In [1]:
import sqlite3
import pandas as pd
import numpy

In [3]:
#Подключаем SQL базу данных
conn_db = sqlite3.connect("bd/main_bd.sqlite", timeout=10)
c = conn_db.cursor()
c.execute("PRAGMA foreign_keys = ON;")

In [3]:
#ПЕСНИ
mainsongs = pd.read_csv(("rootDataset/songs.csv"), sep=';')
#Удаляем дубликаты
mainsongs = mainsongs.drop_duplicates(keep='first', ignore_index=True)


In [20]:

#Реестр всех жанров
GENRES_REG = set(mainsongs["genre_name"])
#reestr = open("bd/genresReg.txt", "w")
#reestr.write((",").join(GENRES_REG))
#reestr.close()

In [4]:
#ДИФФЕРЕНЦИРУЕМ ТАБЛИЦЫ
#ПЕСНИ
songs = mainsongs[['spotify_id','name', 'artist']].drop_duplicates(subset=['spotify_id'], keep='first', ignore_index=True)
songs.insert(0,"songId", songs.index)

In [5]:
#ЖАНРЫ
"""
#SZIP = zip(songs['songid'],songs['name'],songs['artist'])
def firstlocsongs(name, artist):
    for i in zip(songs['songid'],songs['name'],songs['artist']):
        if i[1] == name and i[2] == artist:
            #print("yes")
            return i[0]
indexs = []#[-1] * len(mainsongs)
#allind = list(mainsongs.index)
genres = mainsongs.iloc[:,1:]
#genres['songid'] = [firstlocsongs(i[0],i[1]) for i in zip(mainsongs['name'], mainsongs['artist'])]
for i in zip(mainsongs['name'], mainsongs['artist']):
    res = firstlocsongs(i[0],i[1])
    #if res == -1: res = indexs[-1]
    indexs.append(res)
    print(res)
    #print(firstlocsongs(i[0],i[1]))
"""
indexs = []#[-1] * len(mainsongs)
j = 0
l = len(mainsongs)
for i in zip(songs['songId'],songs['spotify_id']):
    while j < l and mainsongs.loc[j]['spotify_id'] == i[1]:
        indexs.append(i[0])
        j += 1

In [14]:
songs = songs.drop(columns=["spotify_id"])

In [12]:
#ЖАНРЫ
genres = mainsongs[['position','genre_name']]
genres.insert(0,"songId",indexs)

    songid  position               genre_name
0        0       195               vapor soul
1        1       111                dixieland
2        2       175       swedish indie rock
3        3        47  sertanejo universitario
4        4         5              faroese pop
5        5       119                steampunk
6        6       142               slash punk
7        7       169             electroclash
8        8       176               slovak pop
9        9       124            southern rock
10       9         1     modern southern rock


In [25]:
#Создаем и заполняем таблицу songs
c.execute(
    """
    DROP TABLE IF EXISTS songs;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS songs(
        songId INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        artist TEXT NOT NULL
        );
    """
    )
songs.to_sql('songs', conn_db, if_exists='append', index=False)
conn_db.commit()

In [26]:
#ПЕСНИ И ЖАНРЫ
c.execute(
    """
    DROP TABLE IF EXISTS genres;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS genres(
        songId INT NOT NULL,
        position INT NOT NULL,
        genre_name TEXT NOT NULL,
        PRIMARY KEY(songId, genre_name),
        FOREIGN KEY(songId) REFERENCES songs(songId) ON DELETE CASCADE ON UPDATE CASCADE
        );
    """
    )
genres.to_sql('genres', conn_db, if_exists='append', index=False)
conn_db.commit()


In [22]:
#ПОЛЬЗОВАТЕЛИ 
#Создаем SQL таблицу пользователей
c.execute(
    """
    DROP TABLE IF EXISTS users;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS users(
        userId INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NULL UNIQUE,
        password TEXT NULL
        );
    """
    )
conn_db.commit()

In [23]:
#АДМИНИСТРАТОРЫ
c.execute(
    """
    DROP TABLE IF EXISTS admins;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS admins(
        userId INT NOT NULL,
        PRIMARY KEY(userId),
        FOREIGN KEY(userId) REFERENCES users(userId) ON DELETE CASCADE ON UPDATE CASCADE
        );
    """
    )
conn_db.commit()

In [27]:
#ТАБЛИЦА С ИЗБРАННЫМИ ФИЛЬМАМИ ПОЛЬЗОВАТЕЛЕЙ
c.execute(
    """
    DROP TABLE IF EXISTS users_favorites;
    """
    )
c.execute(
    """
    CREATE TABLE IF NOT EXISTS users_favorites(
        userId INT NOT NULL,
        songId INT NOT NULL,
        PRIMARY KEY(userId,songId),
        FOREIGN KEY(userId) REFERENCES users(userId) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY(songId) REFERENCES songs(songId) ON DELETE CASCADE ON UPDATE CASCADE
        );
    """
    )
conn_db.commit()

In [32]:
#ОТОБРАЖЕНИЕ ПЕСЕН: НАЗВАНИЕ, АВТОР, ЖАНРЫ
c.execute(
    """
    DROP VIEW IF EXISTS songs_info;
    """
)
c.execute(
    """
    CREATE VIEW songs_info AS 
    SELECT s.songId AS songId, name, artist, GROUP_CONCAT(genre_name,',') AS genres 
    FROM songs s
    INNER JOIN genres gen ON s.songId = gen.songId
    GROUP BY s.songId
    """
)
conn_db.commit()

In [33]:
#ТРИГГЕР НА users НА INSERT
c.execute(
    """
    CREATE TRIGGER unique_username BEFORE INSERT ON users
    WHEN EXISTS (SELECT * FROM users WHERE LOWER(username) = LOWER(NEW.username))
    BEGIN
        SELECT RAISE(ABORT, 'Unique username');
    END;
    """
)
conn_db.commit()

In [39]:
#ОТОБРАЖЕНИЯ КОЛИЧЕСТВА ПЕСЕН В КАЖДОМ ЖАНРЕ
c.execute(
    """
    DROP VIEW IF EXISTS genres_count;
    """
)
c.execute(
    """
    CREATE VIEW genres_count AS
    SELECT genre_name, MAX(position) as position
    FROM genres
    GROUP BY genre_name
    """
)
conn_db.commit()

In [40]:
conn_db.commit()
conn_db.close()

In [5]:
c.execute(
    """
    DELETE FROM genres WHERE songId = 203842 
    """
)
conn_db.commit()